In [291]:
import os
import pandas as pd
import nltk import word_tokenize, pos_tag, ne_chunk
import re
from mordecai import Geoparser

# Download NLP data for country extraction // make this modular by setting nltk data path
nltk.download('treebank')
nltk.download('maxent_treebank_pos_tagger')
nltk.download('punkt') # Download corpora for GPE extraction
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columns

[nltk_data] Downloading package treebank to
[nltk_data]     /Users/gregory/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /Users/gregory/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /Users/gregory/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/gregory/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/gregory/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /Users/gregory/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [282]:
data = pd.read_csv('../data/raw/titanic.csv')

In [ ]:
def clean_data(data):

    # Calculate size of family including passenger
    data['family_size'] = data['sibsp'] + data['parch'] + 1


    # Add surname column to easily identify relatives. Neo4j can 
    # build relationships based on matching surname and family size.
    data['surname'] = data['name'].str.split(',', expand=True)[0]

    # Extract deck from cabin number
    data['deck'] = data['cabin'].str[:1]
    
    # Fill incorrect NaN values of embarked with correct values for passengers
    data.loc[data['ticket'] == '113572', 'embarked'] = 'S'
    
    # Replace NaN values with Unknown in destination column
    data['home.dest'].fillna("Unknown Destination", inplace=True)
    
    # Save
    data.to_csv('../data/clean/titanic_clean.csv', index=False)
    
    return data

In [ ]:
clean_data(data)

In [ ]:
#data['home.dest'].str.split(' / ', expand=True)[0].value_counts()

In [ ]:
# Create dictionary of Country:[strings]
# If row.`home.dest` is in map assign Country value

countries = {"United States":["NY", "PA", "MA", "MN", "IL", "KS", "OH", "CA", "MO", "DC", "WI", "IA", 
                              "OR", "NJ", "RI", "MI", "ME", "CT", "WV", "ND", "Ohio", "MT", "UT", 
                              "Pennsylvania", "NM", "CO", "SD", "VT", "KY", "FL", "IN", "WA", "AZ", "NE"], 
             "United Kingdom":["London", "England", "Middlesex", "Surrey", "Berkshire", "Guernsey", "Sussex",
                              "Kent", "Essex", "Avon", "Wales", "Weston-Super-Mare", "Cornwall", "Dorset",
                              "Yorks", "Lancashire", "Huntingdonshire", "Somerset", "Glasgow", "Devon", 
                              "Southington", "Staffs", "Aberdeen", "Liverpool", "Southampton", "Belfast", "Norwich"], 
             "France":["Paris", "France"], 
             "Germany":["Germany"], 
             "Austria":["Austria", "Austria-Hungary"], 
             "Italy":["Italy"],
             "Greece":["Greece"], 
             "Ireland":["Ireland", "Co"],
             "Bulgaria":["Bulgaria"], 
             "Canada":["PQ", "ON", "MB", "NS", "BC", "AB", "NB"], 
             "Australia":["Australia"],
             "Russia":["Russia", "Moscow"], 
             "Cuba":["Cuba"], 
             "Sweden":["Sweden"], 
             "Norway":["Norway"], 
             "Denmark":["Denmark"], 
             "Finland":["Finland"], 
             "Mexico":["Mexico"],
             "Croatia":["Croatia"], 
             "Switzerland":["Switzerland"], 
             "Peru":["Peru"], 
             "Belgium":["Belgium"], 
             "India":["India"], 
             "South Africa":["South Africa"], 
             "Uruguay":["Uruguay"], 
             "Spain":["Spain"], 
             "Japan":["Japan"], 
             "Portugal":["Portugal"], 
             "Syria":["Syria"], 
             "Thailand":["Thailand"],
             "Argentina":["Argentina"],
             "Netherlands":["Netherlands"]}


In [283]:
pattern = re.compile('[\W_]+')
sentence = data['home.dest'].str.replace(pattern, ' ')[3]
sentence2 = "I moved from Chicago"

In [284]:
sentence

'Montreal PQ Chesterville ON'

In [285]:
geo = Geoparser()
geo.geoparse('Montreal, Quebec, Chesterville, Ontario')

[{'word': 'Montreal',
  'spans': [{'start': 0, 'end': 8}],
  'country_predicted': 'CAN',
  'country_conf': 0.52780867},
 {'word': 'Quebec',
  'spans': [{'start': 10, 'end': 16}],
  'country_predicted': 'CAN',
  'country_conf': 0.52780867},
 {'word': 'Chesterville',
  'spans': [{'start': 18, 'end': 30}],
  'country_predicted': 'USA',
  'country_conf': 0.73197085,
  'geo': {'admin1': 'South Carolina',
   'lat': '34.70486',
   'lon': '-81.21426',
   'country_code3': 'USA',
   'geonameid': '4574565',
   'place_name': 'Chester',
   'feature_class': 'P',
   'feature_code': 'PPLA2'}},
 {'word': 'Ontario',
  'spans': [{'start': 32, 'end': 39}],
  'country_predicted': 'CAN',
  'country_conf': 0.52780867}]

In [286]:
tokens = word_tokenize(sentence)

In [287]:
tokens

['Montreal', 'PQ', 'Chesterville', 'ON']

In [288]:
pos_list = pos_tag(tokens)

In [289]:
entities = ne_chunk(pos_list)

In [290]:
print(entities)

(S (PERSON Montreal/NNP) PQ/NNP Chesterville/NNP ON/NNP)


In [261]:
home[1][0] in countries['United States']

False

In [262]:
#data['home.dest'].str.contains('Co')

In [263]:
for country in countries:
    if df.str.contains(country[city]):
        df['dest.country'] = countries_dict[country]
        
        
        #for country in countries.keys():
 #   for row in data.iterrows():
  #      for index, destination in home.iteritems():
   #         for idx, word in enumerate(destination):
    #            if word in countries[country]:
     #               #print(index)
      #              data[row, 'country'] = countries[country]

NameError: name 'df' is not defined